In [16]:
import pandas as pd
import numpy as np

In [2]:
dat_label = pd.read_csv('dataset.csv',index_col=0).T
#label = label.iloc[:12,:]
dat_smiles = pd.read_csv('smiles.csv', index_col=0)
s = dat_smiles.to_dict()
dict_smile = s['SMILES']

In [3]:
dat_label.iloc[:,57]

CID100031      0
CID10012081    0
CID10015       0
CID100197      0
CID10024       0
              ..
CID99870       0
CID9989        0
CID9989226     1
CID999         0
CID9990075     0
Name: odorless, Length: 5596, dtype: int64

In [4]:
from rdkit import Chem
from mordred import Calculator, descriptors

# create descriptor calculator with all descriptors
calc = Calculator(descriptors, ignore_3D=True)

len(calc.descriptors)

1613

In [5]:
mols = [Chem.MolFromSmiles(smi) for smi in dict_smile.values()]

In [6]:
df = calc.pandas(mols)

  2%|▉                                       | 132/5596 [00:03<03:43, 24.50it/s]

/home/ubuntu/anaconda/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ubuntu/anaconda/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


  8%|███▏                                    | 446/5596 [00:10<01:39, 51.55it/s]

/home/ubuntu/anaconda/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 15%|█████▊                                  | 821/5596 [00:18<01:28, 53.79it/s]

...

 24%|█████████▍                             | 1358/5596 [00:30<01:31, 46.15it/s]

/home/ubuntu/anaconda/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 36%|█████████████▊                         | 1989/5596 [00:43<03:07, 19.23it/s]

/home/ubuntu/anaconda/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|███████████████████████████████████████| 5596/5596 [01:54<00:00, 48.72it/s]


In [9]:
df.to_csv('mordred.csv')

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [3]:
import torch
import torch.nn.functional as F   # 激励函数的库
from torchvision import datasets
import torchvision.transforms as transforms
import numpy as np
import pandas as pd

In [6]:
df = pd.read_csv('mordred.csv',index_col=0)

/tmp/ipykernel_253031/216529726.py:1: DtypeWarning: Columns (5,6,7,8,9,10,11,12,13,14,15,16,54,55,56,57,58,59,60,61,62,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,136,137,138,139,145,146,147,148,153,154,155,156,157,163,164,165,166,172,173,174,175,181,182,183,184,189,190,191,192,193,198,199,200,201,202,207,208,209,210,211,217,218,219,220,226,227,228,229,234,235,236,237,238,239,240,241,242,261,262,263,264,265,266,267,268,269,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,342,343,344,345,346,352,353,354,355,361,362,363,364,369,370,371,372,373,379,380,381,382,388,389,390,391,397,398,399,400,405,406,407,408,409,414,415,416,417,418,423,424,425,426,427,433,434,435,436,442,443,444,445,450,451,452,453,458,459,460,461,466,467,468,469,474,475,476,477,482,483,484,485,490,491,492,493,498,499,500,501,506,507,508,509,514,515,516,517,522,523,524,525,530,531,532,533,538,539,540,541,546,547

In [10]:
dat_label = pd.read_csv('dataset.csv',index_col=0).T
#label = label.iloc[:12,:]
dat_smiles = pd.read_csv('smiles.csv', index_col=0)
s = dat_smiles.to_dict()
dict_smile = s['SMILES']

In [69]:
X = df.select_dtypes(include=[int, float]).to_numpy()

In [70]:
y = dat_label.to_numpy()

In [71]:
y=torch.tensor(y, dtype=torch.float) 
X=torch.tensor(X, dtype=torch.float)

In [72]:
class Mydatasets(torch.utils.data.Dataset):
    def __init__(self, data1, label ,transform = None):
        self.transform = transform

        self.data1 = data1
        self.label = label

        self.datanum = len(data1)

    def __len__(self):
        return self.datanum

    def __getitem__(self, idx):
        
        out_data1 = torch.tensor(self.data1[idx]).float()
        out_label = torch.tensor(self.label[idx])
        if self.transform:
            out_data1 = self.transform(out_data1)

        return out_data1, out_label

In [73]:
train_dat, test_dat, train_label, test_label = train_test_split(X, y, test_size = 0.1, stratify = None,random_state = 66)    
print('train data:',len(train_dat))
print('test data:',len(test_dat))


train_data_set = Mydatasets(data1 = train_dat, label = train_label)
test_data_set = Mydatasets(data1 = test_dat, label = test_label)

train_loader = torch.utils.data.DataLoader(train_data_set, batch_size = 64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data_set, batch_size = 64, shuffle=False)

train data: 5036
test data: 560


In [74]:
class MLP(torch.nn.Module):   # 继承 torch 的 Module
    def __init__(self):
        super(MLP,self).__init__()    # 
        # 初始化三层神经网络 两个全连接的隐藏层，一个输出层
        self.fc1 = torch.nn.Linear(697,512)  # 第一个隐含层  
        self.fc2 = torch.nn.Linear(512,128)  # 第二个隐含层
        self.fc3 = torch.nn.Linear(128,91)   # 输出层
        
    def forward(self,din):
        # 前向传播， 输入值：din, 返回值 dout
        din = din.view(-1,697)       # 将一个多行的Tensor,拼接成一行
        dout = F.relu(self.fc1(din))   # 使用 relu 激活函数
        dout = F.relu(self.fc2(dout))
        dout = F.softmax(self.fc3(dout), dim=1)  # 输出层使用 softmax 激活函数
        # 10个数字实际上是10个类别，输出是概率分布，最后选取概率最大的作为预测值输出
        return dout

In [75]:
# 训练神经网络
def train():
    #定义损失函数和优化器
    lossfunc = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(params = model.parameters(), lr = 0.01)
    # 开始训练
    for epoch in range(n_epochs):
        train_loss = 0.0
        for data,target in train_loader:
            optimizer.zero_grad()   # 清空上一步的残余更新参数值
            output = model(data)    # 得到预测值
            loss = lossfunc(output,target)  # 计算两者的误差
            loss.backward()         # 误差反向传播, 计算参数更新值
            optimizer.step()        # 将参数更新值施加到 net 的 parameters 上
            train_loss += loss.item()*data.size(0)
        train_loss = train_loss / len(train_loader.dataset)
        print('Epoch:  {}  \tTraining Loss: {:.6f}'.format(epoch + 1, train_loss))
        # 每遍历一遍数据集，测试一下准确率
        test()


In [80]:
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            break

/tmp/ipykernel_253031/4241691745.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  out_data1 = torch.tensor(self.data1[idx]).float()
/tmp/ipykernel_253031/4241691745.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  out_label = torch.tensor(self.label[idx])


In [82]:
labels.shape

torch.Size([64, 91])

In [85]:
outputs.shape

torch.Size([64, 91])

In [114]:
# 在数据集上测试神经网络
def test():
    correct = 0
    total = 0
    with torch.no_grad():  # 训练集中不需要反向传播
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            #_, predicted = torch.max(outputs.data, 1)
            predicted = torch.sigmoid(outputs.data)
            total += labels.size(0) * labels.size(1)
            correct += (torch.round(predicted) == labels).sum().item()
    print('Accuracy of the network on the test images: %d %%' % (
        100 * correct / total))
    return 100.0 * correct / total


In [115]:
# 声明感知器网络
model = MLP()

In [116]:
# 定义全局变量
n_epochs = 10     # epoch 的数目
batch_size = 64  # 决定每次读取多少图片

In [117]:
train()

/tmp/ipykernel_253031/4241691745.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  out_data1 = torch.tensor(self.data1[idx]).float()
/tmp/ipykernel_253031/4241691745.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  out_label = torch.tensor(self.label[idx])


Epoch:  1  	Training Loss: 18.350484
Accuracy of the network on the test images: 94 %
Epoch:  2  	Training Loss: 18.343995
Accuracy of the network on the test images: 94 %
Epoch:  3  	Training Loss: 18.343994
Accuracy of the network on the test images: 94 %
Epoch:  4  	Training Loss: 18.343992
Accuracy of the network on the test images: 94 %
Epoch:  5  	Training Loss: 18.343990
Accuracy of the network on the test images: 94 %
Epoch:  6  	Training Loss: 18.343985
Accuracy of the network on the test images: 94 %
Epoch:  7  	Training Loss: 18.343974
Accuracy of the network on the test images: 94 %
Epoch:  8  	Training Loss: 18.343946
Accuracy of the network on the test images: 94 %
Epoch:  9  	Training Loss: 18.343869
Accuracy of the network on the test images: 94 %
Epoch:  10  	Training Loss: 18.343798
Accuracy of the network on the test images: 94 %
